**Dataset link**
[IMDB Dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

It has 50000 reviews which are classified as positive and negative

**Importing datasets**

In [1]:
# To load in PC install this first
# !pip install -q tensorflow-datasets


import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete90FSNC/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete90FSNC/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


**Splitting Training and Testing Data and converting it to numpy array**

In [2]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


**Preprocess the data and encoding it to tokens**

In [4]:
vocab_size = 10000  # vocabulary size for 10000 unique words
embedding_dim = 16 # dimensions of embedding
max_length = 120   # maxium allowed words per sentence
trunc_type='post'  # longer than 120 words will be truncated and 0 will be placed there
oov_tok = "<OOV>"  # out of volcabulary


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


**Convert the word_index from (key,value) to (value,key)**

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


**Defining the model**
*   vocab_size = 10000
*   embedding_dim = 16
*   input_length = 120

***Instead if flatten() we can also use GlobalAveragePooling1D()***


In [7]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


**Compile the model**

In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])


**Summary of the model**

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


**Training the model**

*   Total epochs = 10



In [10]:
num_epochs = 10

model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 5ms/step - loss: 0.5036 - accuracy: 0.7309 - val_loss: 0.3414 - val_accuracy: 0.8513
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2447 - accuracy: 0.9034 - val_loss: 0.3600 - val_accuracy: 0.8436
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1063 - accuracy: 0.9711 - val_loss: 0.4394 - val_accuracy: 0.8299
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0288 - accuracy: 0.9958 - val_loss: 0.5289 - val_accuracy: 0.8267
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0075 - accuracy: 0.9994 - val_loss: 0.6064 - val_accuracy: 0.8240
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.6562 - val_accuracy: 0.8261
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6901 - val_accuracy: 0.8294
Epoch 

In [12]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # the shape will be: (vocab_size, embedding_dim)

(10000, 16)


**Now visualize it in the embedding projector**

Upload the downloaded files to visualize it:
1.   https://projector.tensorflow.org/



In [16]:
# now iterate through 16 dimensios to pull out values
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# It will download 2 files for you
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [15]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
